# **Banking Analytics Dashboard Project**
## **Steven Sie Santosa**




### **Calculating the categorization for 5 groups of spender based on their Total Spend**
This Python script is used for a partial part of the overall analytics dashboard, which is primarily built using Power BI. The purpose of this script is to perform more accurate data splitting to support the definition of DAX measures for customer spending categories.

Install necessary library

In [ ]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install os


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


Import the necessary library

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

##Read the "fact_spend" documents
this fact_spends is the table data for all of the transaction records of Miltron Bank, in this case we focused on the "spend" column

In [ ]:
df = pd.read_csv("fact_spends.csv")
df


,customer_id,month,category,payment_type,spend
0,ATQCUS1371,July,Health & Wellness,Credit Card,1114
1,ATQCUS0368,October,Groceries,Credit Card,1466
2,ATQCUS0595,May,Health & Wellness,Credit Card,387
3,ATQCUS0667,October,Electronics,Credit Card,1137
4,ATQCUS3477,September,Bills,UPI,2102
...,...,...,...,...,...
863995,ATQCUS1993,June,Bills,Debit Card,897
863996,ATQCUS1063,September,Bills,Credit Card,2680
863997,ATQCUS0416,August,Others,Credit Card,270
863998,ATQCUS3361,September,Bills,UPI,446


Describe the statistical number of the data

In [ ]:
df.describe()

,spend
count,864000.000000
mean,614.464994
std,661.571676
min,6.000000
25%,191.000000
50%,395.000000
75%,793.000000
max,10313.000000


Making the 'Total Spend' of each unique customer ID

In [ ]:
customer_spend_summary = (
    df
    .groupby("customer_id", as_index=False)
    .agg(Total_Spend=("spend", "sum"))
)
customer_spend_summary

,customer_id,Total_Spend
0,ATQCUS0001,170545
1,ATQCUS0002,168514
2,ATQCUS0003,157534
3,ATQCUS0004,162655
4,ATQCUS0005,157296
...,...,...
3995,ATQCUS3996,99589
3996,ATQCUS3997,56264
3997,ATQCUS3998,59760
3998,ATQCUS3999,67142


Finding the distribution of spending for each customer

In [1]:
from matplotlib import pyplot as plt
customer_spend_summary['Total_Spend'].plot(kind='hist', bins=20, title='Total_Spend')
plt.gca().spines[['top', 'right',]].set_visible(False)

NameError: name 'customer_spend_summary' is not defined

Describing the statistical data of the customer spending summary

In [ ]:
customer_spend_summary.describe()


,Total_Spend
count,4000.000000
mean,132724.438750
std,54988.167095
min,35265.000000
25%,90933.750000
50%,120392.500000
75%,163112.500000
max,315201.000000


To support the analysis of customer spending behavior, I first aimed to classify customers into spender categories (e.g., low, medium, high, top spender) based on their total expenditure. However, since the problem statement does not provide any predefined thresholds or criteria for these categories, I established a custom segmentation method. This classification was necessary to later analyze, in Power BI, which occupations are most prevalent in the top spender segment—insight that can guide strategic targeting for the new credit card promotions. Ideally, having labeled data indicating each customer’s spender tier would improve accuracy, but in its absence, I opted to define these categories myself using a data-driven approach.

Since the data distribution is right-skewed and not unimodal, using the range method to split the data (e.g., into equal-width bins) may not effectively handle outliers or provide meaningful groupings. In such cases, applying the quartile method (dividing the data into Q1, Q2, Q3, Q4, Q5 based on percentiles) is more robust. This approach is less sensitive to skewness and provides a more even distribution of data across groups, ensuring each quartile contains an equal number of observations, which improves segmentation accuracy and insight reliability.

In [2]:
customer_spend_summary["Spend_Group"] = pd.qcut(
    customer_spend_summary["Total_Spend"],
    q=5,  # Number of quantile groups
    labels=["Q1", "Q2", "Q3", "Q4", "Q5"]
)

customer_spend_summary["Spend_Category"] = pd.qcut(
    customer_spend_summary["Total_Spend"],
    q=5,
    labels=[
        "Low Spender",
        "Moderate Spender",
        "High Spender",
        "Very High Spender",
        "Top Spender"
    ]
)


customer_spend_summary

NameError: name 'pd' is not defined

Describe the statistical data of each group for further analysis

In [ ]:
customer_spend_summary.groupby("Spend_Group")["Total_Spend"].describe()

/tmp/ipython-input-8-1924911595.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  customer_spend_summary.groupby("Spend_Group")["Total_Spend"].describe()


,count,mean,std,min,25%,50%,75%,max
Spend_Group,,,,,,,,
Q1,800.0,70550.98500,10249.745800,35265.0,65045.00,72717.0,78405.50,84528.0
Q2,800.0,97125.84750,7066.732328,84532.0,90933.75,97598.0,103228.25,108878.0
Q3,800.0,120614.28000,6978.623973,108893.0,114617.00,120392.5,126158.50,133743.0
Q4,800.0,152935.09125,12566.015624,133920.0,141810.00,151909.0,163112.50,177259.0
Q5,800.0,222395.99000,32030.202771,177308.0,194551.00,217708.5,245457.00,315201.0


This provides key descriptive statistics—particularly the minimum and maximum values—for each group. Understanding these values is essential for defining precise thresholds that determine which spend group each customer falls into. These thresholds can then be used in Power BI through DAX functions to dynamically categorize customers based on their total spending. This method ensures consistency between the Python-based analysis and the Power BI dashboard, and enables scalable, rule-based segmentation for future credit card targeting strategies.